In [1]:
import pyactr as actr

aBoxCon = actr.ACTRModel(
    automatic_visual_search=False,
    motor_prepared=True,
    subsymbolic=True,
    utility_noise=0.2,
    partial_matching=False,
    utility_learning=True,
    production_compilation=True,
    activation_trace=True,
    retrieval_threshold=0,
    decay=0
)

aBoxCon.goals = {}
aBoxCon.set_goal("g")
aBoxCon.set_goal("imaginal")
aBoxCon.set_goal("imaginal_action")

actr.chunktype("goal", "state")
actr.chunktype("proposition", "thing, form, element, mainconnective, relation, subformula1, inferred1, subformula2, inferred2, derived")
actr.chunktype("checklist", "thing, form, element, mainconnective, relation, subformula1, subformula2, form2, form3, form4, form5, form6, form7, form8")
actr.chunktype("storelist", "thing, form, element, mainconnective, relation, subformula1, subformula2, derived, inferred1, inferred2, form2, form3, form4, form5, form6, form7, form8, form9, form10, form11, form12, form13, form14, form15")


dm = aBoxCon.decmem
goal = aBoxCon.goals["g"]
imaginal = aBoxCon.goals["imaginal"]
imaginal_action = aBoxCon.goals["imaginal_action"]
retrieval = aBoxCon.retrieval
production = aBoxCon.productions

aBoxCon.goals["g"].add(actr.makechunk(typename="goal", state="start"))
aBoxCon.goals["imaginal"].add(actr.makechunk(typename="checklist", thing="checklist"))

In [2]:
abox = [
"graham:teacher&(supervisor&headofresearch)",
"(tjeerd,fredrik):supervisedby",
"tjeerd:student&employee",
"fredrik:teacher&supervisor",
"tjeerd:student"]

In [3]:
def findBrackets( aString ):
   if '(' in aString:
      match = aString.split('(',1)[1]
      open = 1
      for index in range(len(match)):
         if match[index] in '()':
            open = (open + 1) if match[index] == '(' else (open - 1)
         if not open:
            return match[:index]
print(findBrackets("teacher"))

None


In [4]:
def makeNiceList ( aList ):
    newList = []
    for i in aList:
        if i!="":
            if i[0]=="&":
                j = i[1:]
                newList.append(j)
            elif i[-1]=="&":
                j = i[:-1]
                newList.append(j)
            else:
                print("Formula has wrong syntax.")
        else:
            continue
    return newList

In [5]:
def unNest1( aString ):
    if "(" in aString:
        subconcept1 = findBrackets(aString)
        subconcept2 = aString.split("({})".format(subconcept1))
        subConceptList = makeNiceList(subconcept2)
        subConceptList.append(subconcept1)
    else:
        subConceptList = aString.split("&")
    return subConceptList

In [6]:
def findAllSubconcepts ( formula ):
    divide = formula.split(":")
    element = divide[0]
    concept = divide[1]
    subConceptListBegin = {concept}
    subConceptListEnd = {}
    while subConceptListBegin != subConceptListEnd:
        subConceptListEnd = subConceptListBegin
        for i in subConceptListBegin:
            subConceptListBegin = subConceptListBegin | set(unNest1(i))
    subConceptListEnd = list(subConceptListEnd)
    return element, subConceptListEnd

In [16]:
def loadFormulaIntoActR (formula, model ):
    decmem = model.decmem
    element, subconcepts = findAllSubconcepts ( formula )
    for i in subconcepts:
        subs = unNest1(i)
        
        if f"{element}:{i}" == formula:
            word = "yes"
        else:
            word = "no"
        
        if len(subs) == 1:
            if i[0]=="n":
                mc = "negation"
                decmem.add(actr.makechunk(typename="proposition",
                                  thing="proposition",
                                  form=f"{element}:{i}",
                                  element=element,
                                  mainconnective=mc,
                                  subformula1=subs[0][1:],
                                  derived=word))
            else:
                mc = "concept"
                decmem.add(actr.makechunk(typename="proposition",
                                  thing="proposition",
                                  form=f"{element}:{i}",
                                  element=element,
                                  mainconnective=mc,
                                  subformula1=subs[0],
                                  derived=word))
        else:
            mc = "conjunction"
            decmem.add(actr.makechunk(typename="proposition",
                                  thing="proposition",
                                  form=f"{element}:{i}",
                                  element=element,
                                  mainconnective=mc,
                                  subformula1=subs[0],
                                  subformula2=subs[1],
                                  derived=word,
                                  inferred1="no",
                                  inferred2="no"))


In [17]:
def loadABox(abox, model):
    for i in abox:
        loadFormulaIntoActR (i, model)

loadABox(abox,aBoxCon)
dm

graham:supervisor
graham:teacher&(supervisor&headofresearch)
graham:supervisor&headofresearch
graham:teacher&(supervisor&headofresearch)
graham:headofresearch
graham:teacher&(supervisor&headofresearch)
graham:teacher&(supervisor&headofresearch)
graham:teacher&(supervisor&headofresearch)
graham:teacher
graham:teacher&(supervisor&headofresearch)
(tjeerd,fredrik):supervisedby
(tjeerd,fredrik):supervisedby
tjeerd:student
tjeerd:student&employee
tjeerd:employee
tjeerd:student&employee
tjeerd:student&employee
tjeerd:student&employee
fredrik:teacher&supervisor
fredrik:teacher&supervisor
fredrik:supervisor
fredrik:teacher&supervisor
fredrik:teacher
fredrik:teacher&supervisor
tjeerd:student
tjeerd:student


{proposition(derived= no, element= graham, form= graham:supervisor, inferred1= , inferred2= , mainconnective= concept, relation= , subformula1= supervisor, subformula2= , thing= proposition): array([0., 0., 0., 0., 0.]), proposition(derived= no, element= graham, form= graham:supervisor&headofresearch, inferred1= no, inferred2= no, mainconnective= conjunction, relation= , subformula1= supervisor, subformula2= headofresearch, thing= proposition): array([0., 0., 0., 0., 0.]), proposition(derived= no, element= graham, form= graham:headofresearch, inferred1= , inferred2= , mainconnective= concept, relation= , subformula1= headofresearch, subformula2= , thing= proposition): array([0., 0., 0., 0., 0.]), proposition(derived= no, element= graham, form= graham:teacher&(supervisor&headofresearch), inferred1= no, inferred2= no, mainconnective= conjunction, relation= , subformula1= teacher, subformula2= supervisor&headofresearch, thing= proposition): array([0., 0.]), proposition(derived= no, elemen